In [8]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
#### 머신러닝을 활용한 데이터 분석
# 정확도 정밀도 재현율
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GridSearchCV
# 분류모델 테스트
from sklearn.metrics import confusion_matrix,precision_recall_curve
from sklearn.metrics import roc_curve,roc_auc_score
# 데이터셋 분류: 트레인테스트셋분류,k-fold,s-kfold, gridsearchcv
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
#ROC AUC 오차행렬
from sklearn.tree import export_graphviz
import graphviz
# 경고 무시 LogisticRegression solver 및 maxiter 문제 
import warnings
warnings.filterwarnings('ignore')

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [66]:
def modelacc(title,model,X_train,y_train,X_test,y_test):
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    acc=np.round(accuracy_score(pred,y_test),4)
    f1=np.round(f1_score(pred,y_test),4)
    pre=np.round(precision_score(pred,y_test),4)
    rec=np.round(recall_score(pred,y_test),4)
    pp=model.predict_proba(X_test)[:,1]
    auc=np.round(roc_auc_score(y_test,pp),4)
    res={'model':title,'accuracy':acc,'precision':pre,'recall':rec,'f1':f1,'roc_auc':auc}
    print('정확도:',res)
    return res

In [63]:
from sklearn.datasets import load_breast_cancer
breast= load_breast_cancer()
breast.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [64]:
X=breast['data']
Y=breast['target']

* 데이터 컬럼정보
    - radius : 반경 
    - texture: 질감 
    - perimeter:  둘레
    - area: 면적
    - smoothness : 매끄러움
    - compactness : 작은정도
    - concavity: 오목함
    - concave points : 오목한 숫자
    - symmetry: 대칭성
    - fractal dimension : 프랙탈 차원
        

In [38]:
brdf=pd.DataFrame(data=X,columns=breast['feature_names'])
brdf['target']=Y
brdf.tail()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0
568,7.76,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.00000,0.1587,0.05884,...,30.37,59.16,268.6,0.08996,0.06444,0.0000,0.0000,0.2871,0.07039,1


In [49]:
X_train,X_test,y_train,y_test= train_test_split(X,Y,random_state=123)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(426, 30) (143, 30) (426,) (143,)


In [61]:
y_test

array([1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0])

In [75]:
modelres=[]

In [76]:
model=DecisionTreeClassifier(random_state=123)
res=modelacc('DT',model,X_train,y_train,X_test,y_test)
modelres.append(res)

model=KNeighborsClassifier( n_neighbors=3)
res=modelacc('KNN',model,X_train,y_train,X_test,y_test)
modelres.append(res)

model=LogisticRegression(random_state=123)
res=modelacc('LR',model,X_train,y_train,X_test,y_test)
modelres.append(res)

model=RandomForestClassifier(random_state=123)
res=modelacc('RF',model,X_train,y_train,X_test,y_test)
modelres.append(res)

model=GradientBoostingClassifier(random_state=123)
res=modelacc('GB',model,X_train,y_train,X_test,y_test)
modelres.append(res)

정확도: {'model': 'DT', 'accuracy': 0.965, 'precision': 0.9775, 'recall': 0.9667, 'f1': 0.9721, 'roc_auc': 0.961}
정확도: {'model': 'KNN', 'accuracy': 0.9441, 'precision': 0.9663, 'recall': 0.9451, 'f1': 0.9556, 'roc_auc': 0.9778}
정확도: {'model': 'LR', 'accuracy': 0.986, 'precision': 1.0, 'recall': 0.978, 'f1': 0.9889, 'roc_auc': 0.9963}
정확도: {'model': 'RF', 'accuracy': 0.993, 'precision': 1.0, 'recall': 0.9889, 'f1': 0.9944, 'roc_auc': 0.9944}
정확도: {'model': 'GB', 'accuracy': 0.979, 'precision': 0.9888, 'recall': 0.9778, 'f1': 0.9832, 'roc_auc': 0.9915}


In [77]:
#XGB

model=XGBClassifier( random_state=123)
res=modelacc('XGB',model,X_train,y_train,X_test,y_test)
modelres.append(res)


[10:32:41] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
정확도: {'model': 'XGB', 'accuracy': 0.972, 'precision': 0.9888, 'recall': 0.967, 'f1': 0.9778, 'roc_auc': 0.9946}


In [78]:
#LGBM
model =LGBMClassifier(n_estimators=100,
                      learning_rate=0.1,
                      max_depth=20,
                      min_child_samples=20,
                      num_leaves=31
                     )
res=modelacc('LGB',model,X_train,y_train,X_test,y_test)
modelres.append(res)

정확도: {'model': 'LGB', 'accuracy': 0.986, 'precision': 0.9888, 'recall': 0.9888, 'f1': 0.9888, 'roc_auc': 0.9948}


In [79]:

modeldf=pd.DataFrame(modelres)
modeldf


,model,accuracy,precision,recall,f1,roc_auc
0,DT,0.9650,0.9775,0.9667,0.9721,0.9610
1,KNN,0.9441,0.9663,0.9451,0.9556,0.9778
2,LR,0.9860,1.0000,0.9780,0.9889,0.9963
3,RF,0.9930,1.0000,0.9889,0.9944,0.9944
4,GB,0.9790,0.9888,0.9778,0.9832,0.9915
5,XGB,0.9720,0.9888,0.9670,0.9778,0.9946
6,LGB,0.9860,0.9888,0.9888,0.9888,0.9948
